In [1]:
import sys
sys.path.append('/home/thiago/Documentos/GitHub/pipeline_de_dados/utils')
from spark_config import SparkConfig
from pyspark.sql import functions as F

In [2]:
spark = SparkConfig()

In [3]:
spark= SparkConfig().spark_config()

24/08/03 12:49:04 WARN Utils: Your hostname, thiago-550XDA resolves to a loopback address: 127.0.1.1; using 192.168.0.62 instead (on interface wlo1)
24/08/03 12:49:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/thiago/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/thiago/.ivy2/cache
The jars for the packages stored in: /home/thiago/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46603cec-48cd-40e8-a08b-42fef6d8a65e;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 141ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

In [6]:
df = spark.read.format('delta').load('s3a://datalake-test-thiago/03-gold/delta/store_sales')

In [7]:
df.show(5)

+-----+-----------+----------+-----+---------+----+-----+-------------+--------------+----------+----------+--------------------+----------------------------+---------------------------+------+-----------------+-----------------+---------------+---------+--------+----+-----+---+------------+---------+-----------------+----------------------+-----------+---------------+
|store|day_of_week|      date|sales|customers|open|promo|state_holiday|school_holiday|store_type|assortment|competition_distance|competition_open_since_month|competition_open_since_year|promo2|promo2_since_week|promo2_since_year| promo_interval|month_map|is_promo|year|month|day|week_of_year|year_week|competition_since|competition_time_month|promo_since|promo_time_week|
+-----+-----------+----------+-----+---------+----+-----+-------------+--------------+----------+----------+--------------------+----------------------------+---------------------------+------+-----------------+-----------------+---------------+---------+-

In [8]:
df.dtypes

[('store', 'int'),
 ('day_of_week', 'int'),
 ('date', 'date'),
 ('sales', 'int'),
 ('customers', 'int'),
 ('open', 'int'),
 ('promo', 'int'),
 ('state_holiday', 'string'),
 ('school_holiday', 'int'),
 ('store_type', 'string'),
 ('assortment', 'string'),
 ('competition_distance', 'double'),
 ('competition_open_since_month', 'int'),
 ('competition_open_since_year', 'int'),
 ('promo2', 'int'),
 ('promo2_since_week', 'int'),
 ('promo2_since_year', 'int'),
 ('promo_interval', 'string'),
 ('month_map', 'string'),
 ('is_promo', 'int'),
 ('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('week_of_year', 'int'),
 ('year_week', 'string'),
 ('competition_since', 'date'),
 ('competition_time_month', 'int'),
 ('promo_since', 'date'),
 ('promo_time_week', 'int')]